In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from geojson import GeoJSON
import json
import matplotlib
from numpy import mean, all, median
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

from bokeh.models import (ColumnDataSource, GMapPlot, Range1d, ColumnDataSource, LinearAxis,
HoverTool, PanTool, WheelZoomTool, BoxZoomTool, ResetTool,CustomJS,
GMapOptions, NumeralTickFormatter, PrintfTickFormatter, MapOptions)
from bokeh.models.glyphs import Patches
from bokeh.plotting import figure, save, show
from bokeh.resources import CDN
from bokeh.io import output_file, show, saving
from bokeh.models.widgets import RadioButtonGroup
from bokeh.layouts import widgetbox, column
from bokeh.models.widgets import Panel, Tabs
from bokeh import tile_providers
from bokeh.tile_providers import CARTODBPOSITRON
import pyproj
outfp = r"./docs/index.html"
output_file(outfp)
# output_notebook() # this command doesn't work in jupyyer labs, only jupyter notebook

df_wash = pd.read_pickle('Washington_2.pickle')
df_wash.head()

df = df_wash[df_wash['total_score'].apply(len)>0].copy()
# This will discard all the hexes where there aren't any observations.
# remember to make this a copy or else you will end up with warnings about writing to a slice
df.head()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

,grid,total_score,bike_score,car_score,mass_score,ride_score,points_list,Y,X
hex_key,,,,,,,,,
79404,"(3, -3, 0)",[100],[39.45681897512268],[12.990695117258923],[226.75359358506427],[21.0],[242034],"[4707925.971892506, 4707973.851839005, 4707973...","[-8575711.420947943, -8575739.064447943, -8575..."
81798,"(-3, 3, 0)",[100],[43.40236093237809],[0.0],[156.4847602046781],[21.0],[243140],"[4708213.251571501, 4708261.131518, 4708261.13...","[-8576209.003947943, -8576236.647447944, -8576..."
80604,"(3, 0, -3)",[100],[42.26390640730412],[16.741951275257325],[92.78495513582726],[21.0],[279253],"[4708213.251571501, 4708261.131518, 4708261.13...","[-8575711.420947943, -8575739.064447943, -8575..."
81802,"(1, 3, -4)",[100],[102.21797524625919],[18.501570039901363],[159.9839344716706],[21.0],[241742],"[4708404.771357496, 4708452.651303995, 4708452...","[-8575877.281947942, -8575904.925447943, -8575..."
82197,"(-4, 4, 0)",[100],[41.431725365382306],[0.0],[153.12321089304797],[21.0],[241284],"[4708261.131518, 4708309.011464499, 4708309.01...","[-8576291.934447942, -8576319.577947943, -8576..."


In [2]:
colors = [
'#ffffcc',
'#d9f0a3',
'#addd8e',
'#78c679',
'#31a354',
'#006837'
]




descriptions = ['Bad','Poor','Average','Good','Great','Excellent']
bins1 = [-1,50,60,70,80,90,200]
bins2 = [-1,10,20,30,40,50,200]
columns = ['total_mean']
metrics = [mean, median]

def summerizer(df, column, function, bins):
    string0 = column + "_" + function.__name__
    df[string0] = df[column].progress_apply(function)
    string1 = string0 + "_color"
    string2 = string0 + "_rating"
    df[string1] = pd.cut(df[string0], bins, labels=colors)
    df[string2] = pd.cut(df[string0], bins, labels=descriptions)
        
    return string0, string1, string2


summerizer(df, 'total_score', mean, bins1)    
summerizer(df, 'total_score', median, bins1)  

summerizer(df, 'car_score', mean, bins2)    
summerizer(df, 'car_score', median, bins2)    


summerizer(df, 'bike_score', mean, bins2)    
summerizer(df, 'bike_score', median, bins2)    

summerizer(df, 'mass_score', mean, bins2)    
summerizer(df, 'mass_score', median, bins2)     

summerizer(df, 'ride_score', mean, bins2)    
summerizer(df, 'ride_score', median, bins2)        

# Okay, it only took me five times to realize I should put a backup point here.  Progress?
df.to_pickle("summerized.pickle")
df.head(1)

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

HBox(children=(IntProgress(value=0, max=6668), HTML(value='')))

,grid,total_score,bike_score,car_score,mass_score,ride_score,points_list,Y,X,total_score_mean,...,mass_score_mean_rating,mass_score_median,mass_score_median_color,mass_score_median_rating,ride_score_mean,ride_score_mean_color,ride_score_mean_rating,ride_score_median,ride_score_median_color,ride_score_median_rating
hex_key,,,,,,,,,,,,,,,,,,,,,
79404,"(3, -3, 0)",[100],[39.45681897512268],[12.990695117258923],[226.75359358506427],[21.0],[242034],"[4707925.971892506, 4707973.851839005, 4707973...","[-8575711.420947943, -8575739.064447943, -8575...",100.0,...,NaN,226.753594,NaN,NaN,21.0,#addd8e,Average,21.0,#addd8e,Average


In [3]:
dsource = ColumnDataSource(df)

wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator

bound = 15000 
washington_y = 38.904978
washington_x = -77.039658
        
x, y = pyproj.transform(wgs84, webdb, washington_x, washington_y)
x_scroll = Range1d(x-bound,x+bound) # I want these to be shared by all the tabs so I am initializing them here
y_scroll = Range1d(y-bound,y+bound) # I want these to be shared by all the tabs so I am initializing them here


class plot_pipeline:
    def __init__(self, source, center, tool_tips):
        self.source = source
        self.tool_tips = tool_tips
        self.make_fig(center[0],center[1])
    
    def make_fig(self, lat, lon):
        
        x, y = pyproj.transform(wgs84, webdb, lon, lat)
        wheel = WheelZoomTool()
        tool_box = [HoverTool(tooltips=self.tool_tips), wheel, PanTool(),
                 BoxZoomTool(), ResetTool()]
        bound = 15000 # 15 km
        plot = figure(x_range=x_scroll,
            y_range=y_scroll,
            toolbar_location="right",
            tools = tool_box, plot_width=800,
            plot_height=800)
        
        plot.toolbar.active_scroll = wheel
        plot.add_tile(CARTODBPOSITRON)
        self.plot=plot
        
    def add_layer(self, column_name):       
        
        hexes = Patches(xs='X', ys='Y',
            fill_color=column_name, fill_alpha=0.4, 
            line_color="black", line_width=0.10)
        
        self.plot.add_glyph(dsource, hexes)
        
    def out(self):
        show(self.plot)
        
    def return_plot(self):
        return self.plot
        
        
wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator





In [4]:
tips = [
        ('Total Average', "@total_score_mean"),
        ('Description', "@total_score_mean_rating"),
        ('Total Median', "@total_score_median"),
        ('Description', "@total_score_median_rating"),

        ('Bike Median', "@bike_score_median"),
        ('Description', "@bike_score_median_rating"),

        ('Car Median', "@car_score_median"),
        ('Description', "@car_score_median_rating"),

        ('Mass Median', "@mass_score_median"),
        ('Description', "@mass_score_median_rating"),

        ('Ride Median', "@ride_score_median"),
        ('Description', "@ride_score_median_rating"),
]
        
wash = plot_pipeline(ColumnDataSource(df), (washington_y, washington_x), tips)
wash.add_layer('total_score_median_color')

# wash.out()


In [5]:
tips2 = [
        ('Bike Average', "@bike_score_mean"),
        ('Description', "@bike_score_mean_rating"),

        ('Bike Median', "@bike_score_median"),
        ('Description', "@bike_score_median_rating"),

]

wash_bike = plot_pipeline(ColumnDataSource(df), (washington_y, washington_x), tips2)
wash_bike.add_layer('bike_score_median_color')

In [6]:
tips3 = [
        ('Car Share Average', "@car_score_mean"),
        ('Description', "@car_score_mean_rating"),

        ('Car Share Median', "@car_score_median"),
        ('Description', "@car_score_median_rating"),

]

wash_car = plot_pipeline(ColumnDataSource(df), (washington_y, washington_x), tips3)
wash_car.add_layer('car_score_median_color')

In [7]:
tips4 = [
        ('Mass Transit Average', "@mass_score_mean"),
        ('Description', "@mass_score_mean_rating"),

        ('Mass Transit Median', "@mass_score_median"),
        ('Description', "@mass_score_median_rating"),

]

wash_mass = plot_pipeline(ColumnDataSource(df), (washington_y, washington_x), tips4)
wash_mass.add_layer('mass_score_median_color')

In [8]:
tips5 = [
        ('Ride Share Average', "@ride_score_mean"),
        ('Description', "@ride_score_mean_rating"),

        ('Ride Share Median', "@ride_score_median"),
        ('Description', "@ride_score_median_rating"),

]

wash_ride = plot_pipeline(ColumnDataSource(df), (washington_y, washington_x), tips5)
wash_ride.add_layer('ride_score_median_color')

In [9]:
tab1 = Panel(child=wash.return_plot(), title="Overall")
tab2 = Panel(child=wash_bike.return_plot(), title="Bike Share")
tab3 = Panel(child=wash_car.return_plot(), title="Car Share")
tab4 = Panel(child=wash_mass.return_plot(), title="Mass Transit")
tab5 = Panel(child=wash_ride.return_plot(), title="Ride Share")


tab = Tabs(tabs=[tab1,tab2, tab3, tab4, tab5])

show(tab)
